In [ ]:
!pip install transformers==4.28.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.0/110.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.3
    Uninstalling transformers-4.52.3:
      Successfully uninstalled transformers-4.52.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.28.1 which is incompatible.


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "gpt2"  # You can choose other GPT-2 variants like "gpt2-medium", "gpt2-large", etc.
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

In [ ]:
# Creating a custom dataset file inside Colab
alien_logs = """
Day 1: The sky cracked open and something massive descended.
Day 2: I found shelter in the ruins of the library. Supplies are low.
Day 3: I made contact. Not all of them are hostile.
Day 4: I decoded a message. They're searching for something... or someone.
Day 5: The power grid is down. I have to move before nightfall.
"""

# Save it as a .txt file
with open("alien_log.txt", "w", encoding="utf-8") as f:
    f.write(alien_logs)

print("alien_log.txt file created!")


alien_log.txt file created!


In [ ]:
with open("alien_log.txt", "r", encoding="utf-8") as f:
    dataset = f.read()


In [ ]:
# This is a conceptual outline, you'll need to adapt it based on your data and specific needs

from transformers import TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import os # Import the os module

# Load your dataset (replace 'your_dataset.txt' with the path to your file)
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="alien_log.txt",
    block_size=32 # Adjust block size as needed - Decreased to 32
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",  # Directory to save the fine-tuned model
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    report_to="none" # Add this argument to disable reporting to any service like wandb
)

# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Start fine-tuning
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=3, training_loss=3.5455780029296875, metrics={'train_runtime': 14.5383, 'train_samples_per_second': 0.413, 'train_steps_per_second': 0.206, 'total_flos': 97984512000.0, 'train_loss': 3.5455780029296875, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./my-fine-tuned-gpt2")
tokenizer.save_pretrained("./my-fine-tuned-gpt2")

('./my-fine-tuned-gpt2/tokenizer_config.json',
 './my-fine-tuned-gpt2/special_tokens_map.json',
 './my-fine-tuned-gpt2/vocab.json',
 './my-fine-tuned-gpt2/merges.txt',
 './my-fine-tuned-gpt2/added_tokens.json')

In [ ]:
prompt = "Day 6: I ventured out to find supplies." # Replace with your desired prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Generate text
output = model.generate(
    input_ids,
    max_length=100,  # Set the maximum length of the generated text
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7
)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Day 6: I ventured out to find supplies. And I went to the store to buy some. I was about to get the other guy and I got a gun.

I go back to my house and find another one. It's a big one and it is. So I go to bed and the next day. The next morning I have a dream and this is it. A dream. Not a nightmare. But it was a night. He had his gun and they are going to


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Remove the leading spaces from these two lines
loaded_model = GPT2LMHeadModel.from_pretrained("./my-fine-tuned-gpt2")
loaded_tokenizer = GPT2Tokenizer.from_pretrained("./my-fine-tuned-gpt2")